<a href="https://colab.research.google.com/github/zackives/upenn-cis5450-hw/blob/main/Interactive_Visualization_in_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StreamLit Data Dashboards

This notebook explores some options for creating data dashboards in Streamlit.  In general, Streamlit is run by creating a local `app.py` file and then running `streamlit app.py` -- it then runs a local web server.

It's based on a sample app from Chris Klose, https://blog.streamlit.io/the-streamlit-agraph-component/.

Given that we want to run on Colab, we need to have the notebook write a .py file and we need to run a *proxy server* that lets us open a connection to the server running on Colab.

In [ ]:
!pip install -q streamlit streamlit_agraph==0.0.28 htbuilder networkx SPARQLWrapper altair

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 695.9/695.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.4/565.4 kB 30.3 MB/s eta 0:00:00


## Set up the CloudFlare proxy

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &

--2025-10-13 21:50:03--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.9.1/cloudflared-linux-amd64 [following]
--2025-10-13 21:50:03--  https://github.com/cloudflare/cloudflared/releases/download/2025.9.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/e30ab3bb-4e6a-464a-8db5-d5cabe6a2f8d?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-10-13T22%3A43%3A07Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-10-13T2

## Create sample dataset

This just writes sample data.

In [ ]:
%%writefile marvel.json
{
 "name": "marvel",
 "img": "http://marvel-force-chart.surge.sh/marvel_force_chart_img/marvel.png",
 "children": [
  {
   "name": "Heroes",
   "children": [
    {
      "hero": "Spider-Man",
      "name": "Peter Benjamin Parker",
      "link": "http://marvel.com/characters/54/spider-man",
      "img":  "http://marvel-force-chart.surge.sh/marvel_force_chart_img/top_spiderman.png",
      "size": 40000
    },
    {
      "hero": "CAPTAIN MARVEL",
      "name": "Carol Danvers",
      "link": "http://marvel.com/characters/9/captain_marvel",
      "img":  "http://marvel-force-chart.surge.sh/marvel_force_chart_img/top_captainmarvel.png",
      "size": 40000
    },
    {
      "hero": "HULK",
      "name": "Robert Bruce Banner",
      "link": "http://marvel.com/characters/25/hulk",
      "img":  "http://marvel-force-chart.surge.sh/marvel_force_chart_img/top_hulk.png",
      "size": 40000
    },
    {
      "hero": "Black Widow",
      "name": "Natalia 'Natasha' Alianovna Romanova",
      "link": "http://marvel.com/characters/6/black_widow",
      "img":  "http://marvel-force-chart.surge.sh/marvel_force_chart_img/top_blackwidow.png",
      "size": 40000
    },
    {
      "hero": "Daredevil",
      "name": "Matthew Michael Murdock",
      "link": "http://marvel.com/characters/11/daredevil",
      "img":  "http://marvel-force-chart.surge.sh/marvel_force_chart_img/top_daredevil.png",
      "size": 40000
    },
    {
      "hero": "Wolverine",
      "name": "James Howlett",
      "link": "http://marvel.com/characters/66/wolverine",
      "img":  "http://marvel-force-chart.surge.sh/marvel_force_chart_img/top_wolverine.png",
      "size": 40000
    },
    {
      "hero": "Captain America",
      "name": "Steven Rogers",
      "link": "http://marvel.com/characters/8/captain_america",
      "img":  "http://marvel-force-chart.surge.sh/marvel_force_chart_img/top_captainamerica.png",
      "size": 40000
    },
    {
      "hero": "Iron Man",
      "name": "Anthony Edward 'Tony' Stark",
      "link": "http://marvel.com/characters/29/iron_man",
      "img":  "http://marvel-force-chart.surge.sh/marvel_force_chart_img/top_ironman.png",
      "size": 40000
    },
    {
      "hero": "THOR",
      "name": "Thor Odinson",
      "link": "http://marvel.com/characters/60/thor",
      "img":  "http://marvel-force-chart.surge.sh/marvel_force_chart_img/top_thor.png",
      "size": 40000
    }
  ]
  },
  {
   "name": "Villains",
   "children": [
    {
      "hero": "Dr. Doom",
      "name": "Victor von Doom",
      "link": "http://marvel.com/characters/13/dr_doom",
      "img":  "http://marvel-force-chart.surge.sh/marvel_force_chart_img/drdoom.png",
      "size": 40000
    },
    {
      "hero": "Mystique",
      "name": "Unrevealed",
      "link": "http://marvel.com/characters/1552/mystique",
      "img":  "http://marvel-force-chart.surge.sh/marvel_force_chart_img/mystique.png",
      "size": 40000
    },
    {
      "hero": "Red Skull",
      "name": "Johann Shmidt",
      "link": "http://marvel.com/characters/1901/red_skull",
      "img":  "http://marvel-force-chart.surge.sh/marvel_force_chart_img/redskull.png",
      "size": 40000
    },
    {
      "hero": "Ronan",
      "name": "Ronan",
      "link": "http://marvel.com/characters/49/ronan",
      "img":  "http://marvel-force-chart.surge.sh/marvel_force_chart_img/ronan.png",
      "size": 40000
    },
    {
      "hero": "Magneto",
      "name": "Max Eisenhardt",
      "link": "http://marvel.com/characters/35/magneto",
      "img":  "http://marvel-force-chart.surge.sh/marvel_force_chart_img/magneto.png",
      "size": 40000
    },
    {
      "hero": "Thanos",
      "name": "Thanos",
      "link": "http://marvel.com/characters/58/thanos",
      "img":  "http://marvel-force-chart.surge.sh/marvel_force_chart_img/thanos.png",
      "size": 40000
    },
    {
      "hero": "Black Cat",
      "name": "Felicia Hardy",
      "link": "http://marvel.com/characters/271/black_cat",
      "img":  "http://marvel-force-chart.surge.sh/marvel_force_chart_img/blackcat.png",
      "size": 40000
    }
   ]
  },
  {
   "name": "Teams",
   "children": [
    {
      "hero": "Avengers",
      "name": "",
      "link": "http://marvel.com/characters/68/avengers",
      "img":  "http://marvel-force-chart.surge.sh/marvel_force_chart_img/avengers.png",
      "size": 40000
    },
    {
      "hero": "Guardians of the Galaxy",
      "name": "",
      "link": "http://marvel.com/characters/70/guardians_of_the_galaxy",
      "img":  "http://marvel-force-chart.surge.sh/marvel_force_chart_img/gofgalaxy.png",
      "size": 40000
    },
    {
      "hero": "Defenders",
      "name": "",
      "link": "http://marvel.com/characters/534/defenders",
      "img":  "http://marvel-force-chart.surge.sh/marvel_force_chart_img/defenders.png",
      "size": 40000
    },
    {
      "hero": "X-Men",
      "name": "",
      "link": "http://marvel.com/characters/71/x-men",
      "img":  "http://marvel-force-chart.surge.sh/marvel_force_chart_img/xmen.png",
      "size": 40000
    },
    {
      "hero": "Fantastic Four",
      "name": "",
      "link": "http://marvel.com/characters/69/fantastic_four",
      "img":  "http://marvel-force-chart.surge.sh/marvel_force_chart_img/fantasticfour.png",
      "size": 40000
    },
    {
      "hero": "Inhumans",
      "name": "",
      "link": "http://marvel.com/characters/1040/inhumans",
      "img":  "http://marvel-force-chart.surge.sh/marvel_force_chart_img/inhumans.png",
      "size": 40000
    }
   ]
  }
 ]
}

Writing marvel.json


## This is Our Program!

We start with a Python file to set the HTML styles. Then we have the actual app.

There is a layout script for setting up the HTML CSS styles etc.

In [ ]:
!wget https://raw.githubusercontent.com/ChrisDelClea/Inspirationals/main/layout.py

--2025-10-13 21:50:04--  https://raw.githubusercontent.com/ChrisDelClea/Inspirationals/main/layout.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1856 (1.8K) [text/plain]
Saving to: ‘layout.py’

layout.py           100%[===================>]   1.81K  --.-KB/s    in 0s      

2025-10-13 21:50:04 (25.3 MB/s) - ‘layout.py’ saved [1856/1856]



This is the actual app, which queries in SPARQL against DBpedia for one version of the graph, and reads the JSON file for the other.

In [ ]:
%%writefile app.py
import streamlit as st
from SPARQLWrapper import SPARQLWrapper, JSON
from streamlit_agraph import agraph, TripleStore, Node, Edge, Config
from layout import footer
import json
import pandas as pd
import altair as alt
import numpy as np

@st.cache_data
def get_inspired():
  sparql = SPARQLWrapper("http://dbpedia.org/sparql")

  query_string = """
  SELECT ?name_pe1_en ?rel_en ?name_pe2_en
  WHERE {
    {
         SELECT ?name_p1 ?rel ?name_p2
         WHERE {
             ?p1 a foaf:Person .
             ?p1 dbo:influencedBy ?p2 .
             ?p2 a foaf:Person .
             ?p1 foaf:name ?name_p1 .
             ?p2 foaf:name ?name_p2 .
            dbo:influencedBy rdfs:label ?rel .
            }
         LIMIT 150
    }
    UNION
    {
         SELECT ?name_p1 ?rel ?name_p2
         WHERE {
            ?p1 a foaf:Person .
            ?p1 dbo:influenced ?p2 .
            ?p2 a foaf:Person .
            ?p1 foaf:name ?name_p1 .
            ?p2 foaf:name ?name_p2 .
            dbo:influenced rdfs:label ?rel .
        }
     LIMIT 150
    }
    FILTER ( LANG(?name_p1) = "en" && LANG(?rel) = "en" && LANG(?name_p2) = "en" )
    BIND ( STR(?name_p1) AS ?name_pe1_en )
    BIND ( STR(?rel) AS ?rel_en )
    BIND ( STR(?name_p2) AS ?name_pe2_en )
  }
  """

  sparql.setQuery(query_string)
  sparql.setReturnFormat(JSON)
  results = sparql.query().convert()
  store = TripleStore()
  for result in results["results"]["bindings"]:
    node1 = result["name_pe1_en"]["value"]
    link = result["rel_en"]["value"]
    node2 = result["name_pe2_en"]["value"]
    store.add_triple(node1, link, node2)
  return store

def app():
  st.sidebar.title("Select a Graph")
  query_type = st.sidebar.selectbox("Choose graph: ", ["Influencers", "Superheroes"]) # could add more stuff here later on or add other endpoints in the sidebar.
  config = Config(height=600, width=700, nodeHighlightBehavior=True, highlightColor="#F7A7A6", directed=True,
                  collapsible=True)

  [graph, bar, controls] = st.tabs(['Graphs', 'Bar Chart', 'Controls'])

  with controls:
    st.title("Streamlit Controls")
    st.slider('Hour', 0, 23, 17)
    st.checkbox('Checkbox')

  # Charting example based on https://blog.finxter.com/bar-charts-learning-streamlit-with-bar-charts/
  with bar:
    st.title("Streamlit Charts")
    energy_source = pd.DataFrame({
      "EnergyType": ["Electricity","Gasoline","Natural Gas","Electricity","Gasoline","Natural Gas","Electricity","Gasoline","Natural Gas"],
      "Price ($)":  [150,73,15,130,80,20,170,83,20],
      "Date": ["2022-1-23", "2022-1-30","2022-1-5","2022-2-21", "2022-2-1","2022-2-1","2022-3-1","2022-3-1","2022-3-1"]
      })

    bar_chart = alt.Chart(energy_source).mark_bar().encode(
            x="Month(Date):O",
            y="Sum(Price ($)):Q",
            color="EnergyType:N"
        )
    st.altair_chart(bar_chart, use_container_width=True)

    chart_data = pd.DataFrame( np.random.randn(25, 3), columns=['first', 'second', 'third'])
    st.dataframe(chart_data)
    st.line_chart(chart_data)

  with graph:
    st.title("Streamlit Graphs")
    st.text("(Based on an Example by Christian Klose)")
    if query_type=="Influencers":
      st.subheader("Influencers")
      with st.spinner("Loading data"):
        store = get_inspired()
        st.write("Nodes loaded: " + str(len(store.getNodes())))
      agraph(list(store.getNodes()), (store.getEdges() ), config)

    if query_type=="Superheroes":
      #based on http://marvel-force-chart.surge.sh/
      with open("./marvel.json", encoding="utf8") as f:
        marvel_file = json.loads(f.read())
        marvel_store = TripleStore()
        for sub_graph in marvel_file["children"]:
          marvel_store.add_triple(marvel_file["name"], "has_subgroup", sub_graph["name"], picture=marvel_file["img"])
          for node in sub_graph["children"]:
            node1 = node["hero"]
            link = "belongs_to"
            node2 = sub_graph["name"]
            pic = node["img"]
            marvel_store.add_triple(node1, link, node2, picture=pic)
        agraph(list(marvel_store.getNodes()), (marvel_store.getEdges()), config)

if __name__ == '__main__':
    app()

Writing app.py


## Run our app in Streamlit

In [ ]:
!streamlit run app.py &>/content/logs.txt &

## Make it visible on the Web!

Click on the output URL to open your app. It may take a while.  If you don't get a URL the first time, try re-running this.

In [ ]:
!sleep 10 | grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1 | xargs -I {} echo "Your tunnel url {}"

In [ ]:
!grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1 | xargs -I {} echo "Your tunnel url {}"


Your tunnel url https://navigation-fundamentals-washer-loads.trycloudflare.com
